# 0.0 Imports

In [1]:
import json
import os
import random

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

## 0.1 Loading dataset

In [2]:
path_home = os.path.dirname(os.getcwd())
# Dados de Treino
path_X_traning = os.path.join(path_home, 'data', 'X_training.csv')
path_y_traning = os.path.join(path_home, 'data', 'y_training.csv')

x_training  = pd.read_csv(path_X_traning)
y_training = pd.read_csv(path_y_traning)

# Dados de Validação
path_X_validation = os.path.join(path_home, 'data', 'X_validation.csv')
path_y_validation = os.path.join(path_home, 'data', 'y_validation.csv')

x_validation  = pd.read_csv(path_X_validation)
y_validation = pd.read_csv(path_y_validation)

# Dados de Teste
path_X_test = os.path.join(path_home, 'data', 'X_test.csv')
path_y_test = os.path.join(path_home, 'data', 'y_test.csv')

x_test  = pd.read_csv(path_X_test)
y_test = pd.read_csv(path_y_test)

In [3]:
x_training.head()

,id,customer_type,age,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,...,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,gender_Female,gender_Male,type_of_travel_business_travel,type_of_travel_personal_travel
0,13508,1,0.500000,0.0,0.039580,0.6,0.6,0.6,0.6,1.0,...,0.5,1.00,0.6,0.4,0.000000,0.013848,1.0,0.0,1.0,0.0
1,28874,1,0.243590,0.0,0.205775,0.6,0.4,0.4,0.4,0.6,...,0.5,0.50,0.2,0.6,0.000000,0.000000,0.0,1.0,1.0,0.0
2,21484,0,0.435897,1.0,0.026858,0.6,0.6,0.6,0.2,1.0,...,0.0,1.00,0.6,1.0,0.000000,0.000000,1.0,0.0,1.0,0.0
3,48280,1,0.589744,0.5,0.041397,0.6,1.0,0.6,0.6,0.8,...,0.0,1.00,0.4,0.4,0.029499,0.020772,1.0,0.0,0.0,1.0
4,472,0,0.423077,1.0,0.016559,0.2,0.2,0.2,0.8,0.6,...,1.0,0.75,0.8,0.6,0.021632,0.019782,0.0,1.0,1.0,0.0


## 0.2 Helpe Function

In [4]:

def classifier_evaluation(model_classifier, param, data):
    data_mapping = {
        'training': (x_training, y_training),
        'validation': (x_training, y_training),
        'test': (
                    pd.concat([x_training, x_validation]).reset_index(drop=True),
                    pd.concat([y_training, y_validation]).reset_index(drop=True),
                )
    }

    x_data, y_data = data_mapping.get(data, (None, None))

    if x_data is None or y_data is None:
        raise ValueError("Invalid data type provided. Choose from 'training', 'validation', or 'test'.")

    model = model_classifier(**param)
    model.fit(x_data, y_data.values.ravel())

    predict_mapping = {
        'training': x_data,
        'validation': x_validation,
        'test': x_test
    }

    yhat_model = model.predict(predict_mapping[data])

    metrics_mapping = {
        'training': y_training,
        'validation': y_validation,
        'test': y_test
    }

    metrics = get_metrics(metrics_mapping[data], yhat_model)

    result = create_result(model, param, metrics)

    return result

def get_metrics(y_data, yhat_model):
    metrics = {
        'accuracy': accuracy_score(y_data, yhat_model),
        'precision_score': precision_score(y_data, yhat_model),
        'recall_score': recall_score(y_data, yhat_model),
        'f1_score': f1_score(y_data, yhat_model),
    }
    return metrics

def create_result(model, param, metrics):
    metric = pd.DataFrame({
        'name': model.__class__.__name__,
        **metrics,
        'param': json.dumps(param)
    }, index=[0])

    return metric
    


In [5]:
x, y  = (pd.concat([x_training, x_validation]).reset_index(drop=True), pd.concat([y_training, y_validation]).reset_index(drop=True))

# 2.0 Training

In [6]:
data = 'training'

## 2.1 KNN

In [7]:
resul_knn = pd.DataFrame()
model = KNeighborsClassifier

for i in range(2, 20, 1):
    param = {
        'n_neighbors': i
    }

    result = classifier_evaluation(model, param, data)

    resul_knn = pd.concat([resul_knn,result]).reset_index(drop=True)

In [8]:
resul_knn.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
1,KNeighborsClassifier,0.832186,0.812008,0.797410,0.804643,"{""n_neighbors"": 3}"
0,KNeighborsClassifier,0.840199,1.000000,0.631284,0.773972,"{""n_neighbors"": 2}"
3,KNeighborsClassifier,0.781562,0.755893,0.732563,0.744046,"{""n_neighbors"": 5}"
5,KNeighborsClassifier,0.756312,0.731249,0.692090,0.711131,"{""n_neighbors"": 7}"
2,KNeighborsClassifier,0.784665,0.852670,0.608247,0.710012,"{""n_neighbors"": 4}"
7,KNeighborsClassifier,0.739999,0.713814,0.667844,0.690064,"{""n_neighbors"": 9}"
4,KNeighborsClassifier,0.757471,0.792164,0.597047,0.680904,"{""n_neighbors"": 6}"
9,KNeighborsClassifier,0.729559,0.704652,0.647321,0.674771,"{""n_neighbors"": 11}"
6,KNeighborsClassifier,0.742177,0.762429,0.588488,0.664260,"{""n_neighbors"": 8}"
11,KNeighborsClassifier,0.718968,0.694723,0.627148,0.659208,"{""n_neighbors"": 13}"


## 2.2 Decision Tree

In [9]:
resul_dt = pd.DataFrame()
model = DecisionTreeClassifier

for i in range(2, 20, 1):
    param = {
        'max_depth': i
    }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [10]:
resul_dt.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
17,DecisionTreeClassifier,0.989644,0.992993,0.983041,0.987992,"{""max_depth"": 19}"
16,DecisionTreeClassifier,0.987299,0.991525,0.979063,0.985255,"{""max_depth"": 18}"
15,DecisionTreeClassifier,0.984638,0.990835,0.973559,0.982121,"{""max_depth"": 17}"
14,DecisionTreeClassifier,0.981052,0.986720,0.969327,0.977946,"{""max_depth"": 16}"
13,DecisionTreeClassifier,0.977936,0.985735,0.963027,0.974248,"{""max_depth"": 15}"
12,DecisionTreeClassifier,0.974516,0.981728,0.959049,0.970256,"{""max_depth"": 14}"
11,DecisionTreeClassifier,0.970847,0.977676,0.954531,0.965965,"{""max_depth"": 13}"
10,DecisionTreeClassifier,0.966379,0.976872,0.944794,0.960565,"{""max_depth"": 12}"
9,DecisionTreeClassifier,0.961056,0.969349,0.939863,0.954378,"{""max_depth"": 11}"
8,DecisionTreeClassifier,0.955444,0.963721,0.932290,0.947745,"{""max_depth"": 10}"


## 2.3 Random Forest

In [11]:
resul_rf = pd.DataFrame()
model = RandomForestClassifier
para = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 3, 4, 5]
    }

for i in range(2, 20, 1):
    param = {
        'n_estimators': random.choice(para['n_estimators']),
        'max_depth': random.choice(para['max_depth'])
    }

    result = classifier_evaluation(model, param, data)


    resul_rf = pd.concat([resul_rf,result]).reset_index(drop=True)

In [12]:
resul_rf.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
17,RandomForestClassifier,1.000000,1.000000,1.000000,1.000000,"{""n_estimators"": 300, ""max_depth"": null}"
7,RandomForestClassifier,1.000000,1.000000,1.000000,1.000000,"{""n_estimators"": 300, ""max_depth"": null}"
14,RandomForestClassifier,1.000000,1.000000,1.000000,1.000000,"{""n_estimators"": 200, ""max_depth"": null}"
11,RandomForestClassifier,1.000000,1.000000,1.000000,1.000000,"{""n_estimators"": 300, ""max_depth"": null}"
10,RandomForestClassifier,1.000000,1.000000,1.000000,1.000000,"{""n_estimators"": 300, ""max_depth"": null}"
6,RandomForestClassifier,0.926043,0.920092,0.908235,0.914125,"{""n_estimators"": 200, ""max_depth"": 5}"
4,RandomForestClassifier,0.924705,0.917680,0.907694,0.912660,"{""n_estimators"": 200, ""max_depth"": 5}"
9,RandomForestClassifier,0.924485,0.917960,0.906803,0.912348,"{""n_estimators"": 300, ""max_depth"": 5}"
2,RandomForestClassifier,0.924030,0.920558,0.902603,0.911492,"{""n_estimators"": 100, ""max_depth"": 5}"
1,RandomForestClassifier,0.923064,0.918309,0.902794,0.910485,"{""n_estimators"": 100, ""max_depth"": 5}"


## 2.4 Logistic Regression

In [13]:
resul_rf = pd.DataFrame()
model = LogisticRegression
para = {
        'C': [0.1, 0.5, 1.0],
        'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    }

for i in range(2, 20, 1):
    param = {
        'C': random.choice(para['C']),
        'solver': random.choice(para['solver']),
    }

    result = classifier_evaluation(model, param, data)


    resul_rf = pd.concat([resul_rf,result]).reset_index(drop=True)

/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_

In [14]:
resul_rf.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
7,LogisticRegression,0.876288,0.871866,0.837661,0.854421,"{""C"": 0.1, ""solver"": ""newton-cholesky""}"
1,LogisticRegression,0.876288,0.871866,0.837661,0.854421,"{""C"": 0.1, ""solver"": ""newton-cholesky""}"
10,LogisticRegression,0.876191,0.871738,0.837565,0.854310,"{""C"": 0.1, ""solver"": ""newton-cg""}"
12,LogisticRegression,0.876067,0.871601,0.837406,0.854162,"{""C"": 0.5, ""solver"": ""newton-cg""}"
9,LogisticRegression,0.876081,0.871704,0.837311,0.854161,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
5,LogisticRegression,0.876081,0.871704,0.837311,0.854161,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
14,LogisticRegression,0.876081,0.871704,0.837311,0.854161,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
11,LogisticRegression,0.876081,0.871704,0.837311,0.854161,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
13,LogisticRegression,0.851300,0.837083,0.815642,0.826224,"{""C"": 0.1, ""solver"": ""lbfgs""}"
4,LogisticRegression,0.793257,0.729220,0.831870,0.777170,"{""C"": 0.1, ""solver"": ""liblinear""}"


# 3.0 validation

In [15]:
data = 'validation'

## 3.1 KNN

In [16]:
resul_knn = pd.DataFrame()
model = KNeighborsClassifier

for i in range(2, 20, 1):
    param = {
        'n_neighbors': i
    }

    result = classifier_evaluation(model, param, data)

    resul_knn = pd.concat([resul_knn,result]).reset_index(drop=True)

In [17]:
resul_knn.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
1,KNeighborsClassifier,0.676277,0.627851,0.621278,0.624548,"{""n_neighbors"": 3}"
3,KNeighborsClassifier,0.675665,0.631775,0.603163,0.617138,"{""n_neighbors"": 5}"
5,KNeighborsClassifier,0.666978,0.622708,0.587571,0.604630,"{""n_neighbors"": 7}"
7,KNeighborsClassifier,0.666495,0.624240,0.578959,0.600747,"{""n_neighbors"": 9}"
9,KNeighborsClassifier,0.664436,0.623477,0.569827,0.595446,"{""n_neighbors"": 11}"
11,KNeighborsClassifier,0.661025,0.621220,0.558171,0.588010,"{""n_neighbors"": 13}"
13,KNeighborsClassifier,0.658580,0.619923,0.548445,0.581997,"{""n_neighbors"": 15}"
15,KNeighborsClassifier,0.652273,0.612931,0.536343,0.572085,"{""n_neighbors"": 17}"
17,KNeighborsClassifier,0.649924,0.612049,0.524983,0.565183,"{""n_neighbors"": 19}"
8,KNeighborsClassifier,0.665401,0.647002,0.501596,0.565096,"{""n_neighbors"": 10}"


## 3.2 Decision Tree

In [18]:
resul_dt = pd.DataFrame()
model = DecisionTreeClassifier

for i in range(2, 20, 1):
    param = {
        'max_depth': i
    }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [19]:
resul_dt.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
12,DecisionTreeClassifier,0.952444,0.956036,0.933180,0.944469,"{""max_depth"": 14}"
11,DecisionTreeClassifier,0.951897,0.955770,0.932140,0.943808,"{""max_depth"": 13}"
10,DecisionTreeClassifier,0.951704,0.958614,0.928651,0.943395,"{""max_depth"": 12}"
13,DecisionTreeClassifier,0.951318,0.955155,0.931398,0.943127,"{""max_depth"": 15}"
14,DecisionTreeClassifier,0.950674,0.950687,0.934665,0.942608,"{""max_depth"": 16}"
15,DecisionTreeClassifier,0.950385,0.949634,0.935110,0.942316,"{""max_depth"": 17}"
9,DecisionTreeClassifier,0.950352,0.956725,0.927389,0.941828,"{""max_depth"": 11}"
17,DecisionTreeClassifier,0.949644,0.946043,0.937263,0.941633,"{""max_depth"": 19}"
16,DecisionTreeClassifier,0.949258,0.946866,0.935407,0.941102,"{""max_depth"": 18}"
8,DecisionTreeClassifier,0.947263,0.954371,0.922414,0.938121,"{""max_depth"": 10}"


## 3.3 Random Forest

In [20]:
resul_rf = pd.DataFrame()
model = RandomForestClassifier
para = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 3, 4, 5]
    }

for i in range(2, 20, 1):
    param = {
        'n_estimators': random.choice(para['n_estimators']),
        'max_depth': random.choice(para['max_depth'])
    }

    result = classifier_evaluation(model, param, data)


    resul_rf = pd.concat([resul_rf,result]).reset_index(drop=True)

In [21]:
resul_rf.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
11,RandomForestClassifier,0.965668,0.975829,0.944168,0.959737,"{""n_estimators"": 200, ""max_depth"": null}"
8,RandomForestClassifier,0.964832,0.974322,0.943723,0.958778,"{""n_estimators"": 300, ""max_depth"": null}"
15,RandomForestClassifier,0.964767,0.973882,0.944020,0.958718,"{""n_estimators"": 200, ""max_depth"": null}"
5,RandomForestClassifier,0.927346,0.919542,0.912169,0.915840,"{""n_estimators"": 200, ""max_depth"": 5}"
6,RandomForestClassifier,0.924933,0.919593,0.906006,0.912749,"{""n_estimators"": 300, ""max_depth"": 5}"
1,RandomForestClassifier,0.924805,0.918433,0.907046,0.912704,"{""n_estimators"": 200, ""max_depth"": 5}"
2,RandomForestClassifier,0.924611,0.919406,0.905412,0.912356,"{""n_estimators"": 100, ""max_depth"": 5}"
10,RandomForestClassifier,0.923839,0.921232,0.901329,0.911172,"{""n_estimators"": 100, ""max_depth"": 5}"
14,RandomForestClassifier,0.920429,0.918927,0.895389,0.907006,"{""n_estimators"": 100, ""max_depth"": 5}"
4,RandomForestClassifier,0.912256,0.911508,0.883288,0.897176,"{""n_estimators"": 100, ""max_depth"": 4}"


## 3.4 Logistic Regression

In [22]:
resul_rf = pd.DataFrame()
model = LogisticRegression
para = {
        'C': [0.1, 0.5, 1.0],
        'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    }

for i in range(2, 20, 1):
    param = {
        'C': random.choice(para['C']),
        'solver': random.choice(para['solver']),
    }

    result = classifier_evaluation(model, param, data)


    resul_rf = pd.concat([resul_rf,result]).reset_index(drop=True)

/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/scipy/optimize/_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/utils/optimize.py:74: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i =

In [23]:
resul_rf.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
12,LogisticRegression,0.874320,0.869256,0.835697,0.852146,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
9,LogisticRegression,0.874288,0.869189,0.835697,0.852114,"{""C"": 1.0, ""solver"": ""newton-cg""}"
13,LogisticRegression,0.874288,0.869189,0.835697,0.852114,"{""C"": 1.0, ""solver"": ""newton-cg""}"
1,LogisticRegression,0.874288,0.869189,0.835697,0.852114,"{""C"": 1.0, ""solver"": ""newton-cg""}"
11,LogisticRegression,0.874256,0.869008,0.835845,0.852104,"{""C"": 0.1, ""solver"": ""newton-cg""}"
3,LogisticRegression,0.869977,0.859792,0.836365,0.847917,"{""C"": 1.0, ""solver"": ""lbfgs""}"
16,LogisticRegression,0.869977,0.859792,0.836365,0.847917,"{""C"": 1.0, ""solver"": ""lbfgs""}"
2,LogisticRegression,0.794556,0.731322,0.831391,0.778152,"{""C"": 0.1, ""solver"": ""liblinear""}"
4,LogisticRegression,0.794556,0.731322,0.831391,0.778152,"{""C"": 0.1, ""solver"": ""liblinear""}"
14,LogisticRegression,0.794556,0.731322,0.831391,0.778152,"{""C"": 0.1, ""solver"": ""liblinear""}"


# 4.0 Test

In [24]:
data = 'test'

## 4.1 KNN

In [25]:
resul_knn = pd.DataFrame()
model = KNeighborsClassifier

for i in range(2, 20, 1):
    param = {
        'n_neighbors': i
    }

    result = classifier_evaluation(model, param, data)

    resul_knn = pd.concat([resul_knn,result]).reset_index(drop=True)

In [26]:
resul_knn.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
1,KNeighborsClassifier,0.688449,0.648025,0.635196,0.641546,"{""n_neighbors"": 3}"
3,KNeighborsClassifier,0.685011,0.646223,0.623933,0.634882,"{""n_neighbors"": 5}"
5,KNeighborsClassifier,0.684818,0.648747,0.614782,0.631308,"{""n_neighbors"": 7}"
7,KNeighborsClassifier,0.679373,0.644604,0.600704,0.621880,"{""n_neighbors"": 9}"
9,KNeighborsClassifier,0.677635,0.644430,0.592433,0.617338,"{""n_neighbors"": 11}"
11,KNeighborsClassifier,0.672885,0.640302,0.581258,0.609353,"{""n_neighbors"": 13}"
13,KNeighborsClassifier,0.669988,0.637857,0.574043,0.604270,"{""n_neighbors"": 15}"
15,KNeighborsClassifier,0.668134,0.636096,0.569996,0.601234,"{""n_neighbors"": 17}"
17,KNeighborsClassifier,0.666165,0.635225,0.562341,0.596565,"{""n_neighbors"": 19}"
6,KNeighborsClassifier,0.680609,0.675037,0.525121,0.590716,"{""n_neighbors"": 8}"


## 4.2 Decision Tree

In [27]:
resul_dt = pd.DataFrame()
model = DecisionTreeClassifier

for i in range(2, 20, 1):
    param = {
        'max_depth': i
    }

    result = classifier_evaluation(model, param, data)

    resul_dt = pd.concat([resul_dt,result]).reset_index(drop=True)

In [28]:
resul_dt.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
12,DecisionTreeClassifier,0.955973,0.955781,0.943335,0.949517,"{""max_depth"": 14}"
13,DecisionTreeClassifier,0.954814,0.952910,0.943687,0.948276,"{""max_depth"": 15}"
11,DecisionTreeClassifier,0.953694,0.952950,0.940959,0.946916,"{""max_depth"": 13}"
14,DecisionTreeClassifier,0.953540,0.951805,0.941839,0.946796,"{""max_depth"": 16}"
10,DecisionTreeClassifier,0.952806,0.954802,0.936824,0.945727,"{""max_depth"": 12}"
15,DecisionTreeClassifier,0.952458,0.952088,0.938935,0.945466,"{""max_depth"": 17}"
16,DecisionTreeClassifier,0.952226,0.949414,0.941311,0.945345,"{""max_depth"": 18}"
9,DecisionTreeClassifier,0.951840,0.952424,0.937088,0.944693,"{""max_depth"": 11}"
17,DecisionTreeClassifier,0.951377,0.947010,0.941927,0.944462,"{""max_depth"": 19}"
8,DecisionTreeClassifier,0.947631,0.947350,0.932512,0.939872,"{""max_depth"": 10}"


## 4.3 Random Forest

In [29]:
resul_rf = pd.DataFrame()
model = RandomForestClassifier
para = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 3, 4, 5]
    }

for i in range(2, 20, 1):
    param = {
        'n_estimators': random.choice(para['n_estimators']),
        'max_depth': random.choice(para['max_depth'])
    }

    result = classifier_evaluation(model, param, data)


    resul_rf = pd.concat([resul_rf,result]).reset_index(drop=True)

In [30]:
resul_rf.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
4,RandomForestClassifier,0.965937,0.974473,0.947206,0.960646,"{""n_estimators"": 200, ""max_depth"": null}"
12,RandomForestClassifier,0.965551,0.975138,0.945623,0.960154,"{""n_estimators"": 200, ""max_depth"": null}"
11,RandomForestClassifier,0.965473,0.974273,0.946326,0.960096,"{""n_estimators"": 300, ""max_depth"": null}"
10,RandomForestClassifier,0.965319,0.975384,0.944831,0.959864,"{""n_estimators"": 100, ""max_depth"": null}"
8,RandomForestClassifier,0.964778,0.974145,0.944831,0.959264,"{""n_estimators"": 100, ""max_depth"": null}"
17,RandomForestClassifier,0.927316,0.921130,0.912538,0.916814,"{""n_estimators"": 300, ""max_depth"": 5}"
16,RandomForestClassifier,0.925308,0.919267,0.909723,0.914470,"{""n_estimators"": 200, ""max_depth"": 5}"
14,RandomForestClassifier,0.923493,0.920054,0.904267,0.912092,"{""n_estimators"": 100, ""max_depth"": 5}"
15,RandomForestClassifier,0.910478,0.908148,0.885614,0.896739,"{""n_estimators"": 100, ""max_depth"": 4}"
3,RandomForestClassifier,0.909898,0.913402,0.877959,0.895330,"{""n_estimators"": 100, ""max_depth"": 4}"


## 4.4 Logistic Regression

In [31]:
resul_rf = pd.DataFrame()
model = LogisticRegression
para = {
        'C': [0.1, 0.5, 1.0],
        'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    }

for i in range(2, 20, 1):
    param = {
        'C': random.choice(para['C']),
        'solver': random.choice(para['solver']),
    }

    result = classifier_evaluation(model, param, data)


    resul_rf = pd.concat([resul_rf,result]).reset_index(drop=True)

/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/scipy/optimize/_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
/mnt/0165652C522E8ECA/ProjetosDeProgramacao/ensaio_ml/.venv/lib/python3.12/site-packages/sklearn/utils/optimize.py:74: LineSearchWarning: The line search algorithm did not converge
  r

In [32]:
resul_rf.sort_values('f1_score', ascending=False)

,name,accuracy,precision_score,recall_score,f1_score,param
14,LogisticRegression,0.871780,0.868057,0.834756,0.851081,"{""C"": 1.0, ""solver"": ""newton-cholesky""}"
6,LogisticRegression,0.871780,0.868057,0.834756,0.851081,"{""C"": 1.0, ""solver"": ""newton-cholesky""}"
2,LogisticRegression,0.871703,0.867831,0.834844,0.851018,"{""C"": 1.0, ""solver"": ""newton-cg""}"
8,LogisticRegression,0.871703,0.867831,0.834844,0.851018,"{""C"": 1.0, ""solver"": ""newton-cg""}"
11,LogisticRegression,0.871626,0.867538,0.835020,0.850968,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
12,LogisticRegression,0.871626,0.867538,0.835020,0.850968,"{""C"": 0.5, ""solver"": ""newton-cholesky""}"
9,LogisticRegression,0.871317,0.867307,0.834492,0.850583,"{""C"": 0.1, ""solver"": ""newton-cholesky""}"
13,LogisticRegression,0.871317,0.867307,0.834492,0.850583,"{""C"": 0.1, ""solver"": ""newton-cholesky""}"
15,LogisticRegression,0.848299,0.820643,0.837396,0.828935,"{""C"": 0.1, ""solver"": ""lbfgs""}"
1,LogisticRegression,0.848299,0.820643,0.837396,0.828935,"{""C"": 0.1, ""solver"": ""lbfgs""}"
